1. Создаем сессию Spark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/07/19 23:33:12 WARN Utils: Your hostname, vancomm-pc resolves to a loopback address: 127.0.1.1; using 172.18.109.114 instead (on interface eth0)
24/07/19 23:33:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/19 23:33:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2. Создаем датафрейм продуктов

In [2]:
import itertools
from pyspark.sql import Row

product_ids = itertools.count(1)

products = spark.createDataFrame(
    [
        Row(product_id=next(product_ids), product_name="Apple"),
        Row(product_id=next(product_ids), product_name="Peach"),
        Row(product_id=next(product_ids), product_name="Milk"),
        Row(product_id=next(product_ids), product_name="Cheese"),
        Row(product_id=next(product_ids), product_name="Butter"),
        Row(product_id=next(product_ids), product_name="Candy bar"),
        Row(product_id=next(product_ids), product_name="Chewing gum"),
    ]
)
products.show()

+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|       Apple|
|         2|       Peach|
|         3|        Milk|
|         4|      Cheese|
|         5|      Butter|
|         6|   Candy bar|
|         7| Chewing gum|
+----------+------------+



3. Создаем датафрейм категорий

In [3]:
category_ids = itertools.count(1)

categories = spark.createDataFrame(
    [
        Row(category_id=next(category_ids), category_name="Produce"),
        Row(category_id=next(category_ids), category_name="Dairy"),
        Row(category_id=next(category_ids), category_name="Discount"),
    ]
)
categories.show()

+-----------+-------------+
|category_id|category_name|
+-----------+-------------+
|          1|      Produce|
|          2|        Dairy|
|          3|     Discount|
+-----------+-------------+



4. Создаем датафрейм связей "продукт-категория"

In [4]:
product_categories = spark.createDataFrame(
    [
        Row(product_id=1, category_id=1),
        Row(product_id=2, category_id=1),
        Row(product_id=2, category_id=3),
        Row(product_id=3, category_id=2),
        Row(product_id=4, category_id=2),
        Row(product_id=5, category_id=2),
        Row(product_id=5, category_id=3),
        Row(product_id=7, category_id=3),
    ]
)

5. Соединяем продукты с категориями, используя LEFT JOIN

In [5]:
df = (
    products.join(product_categories, on="product_id", how="left")
    .join(categories, on="category_id", how="left")
    .select(["product_name", "category_name"])
)
df.show()

+------------+-------------+
|product_name|category_name|
+------------+-------------+
|       Apple|      Produce|
|        Milk|        Dairy|
|       Peach|     Discount|
|       Peach|      Produce|
|      Butter|     Discount|
|      Butter|        Dairy|
|      Cheese|        Dairy|
| Chewing gum|     Discount|
|   Candy bar|         NULL|
+------------+-------------+



6. Выводим все пары "имя продукта-имя категории" и имена всех продуктов, у которых нет категорий

In [6]:
from pprint import pprint

products_with_categories, uncategorized_products = (
    df.where(df["category_name"].isNotNull())
    .rdd.map(tuple)
    .collect(),
    df.where(df["category_name"].isNull())
    .select("product_name")
    .toPandas()["product_name"]
    .tolist(),
)

pprint(products_with_categories)
pprint(uncategorized_products)

[('Peach', 'Produce'),
 ('Apple', 'Produce'),
 ('Butter', 'Dairy'),
 ('Cheese', 'Dairy'),
 ('Milk', 'Dairy'),
 ('Chewing gum', 'Discount'),
 ('Butter', 'Discount'),
 ('Peach', 'Discount')]
['Candy bar']
